# Fine-tune a 🤗 Transformers model

This notebook is based on [an official 🤗 notebook - "How to fine-tune a model on text classification"](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb). The main aim of this notebook is to show the process of conversion from vanilla 🤗 to [Ray AIR](https://docs.ray.io/en/latest/ray-air/getting-started.html) 🤗 without changing the training logic unless necessary.

In this notebook, we will:
1. [Set up Ray](#setup)
2. [Load the dataset](#load)
3. [Preprocess the dataset with Ray AIR](#preprocess)
4. [Run the training with Ray AIR](#train)
5. [Predict on test data with Ray AIR](#predict)
6. [Optionally, share the model with the community](#share)

Uncomment and run the following line in order to install all the necessary dependencies (this notebook is being tested with `transformers==4.19.1`):

In [1]:
#! pip install "datasets" "transformers>=4.19.0" "torch>=1.10.0" "mlflow" "ray[air]>=1.13"

## Set up Ray <a name="setup"></a>

We will use `ray.init()` to initialize a local cluster. By default, this cluster will be compromised of only the machine you are running this notebook on. You can also run this notebook on an Anyscale cluster.

Note: this notebook *will not* run in Ray Client mode.

In [2]:
from pprint import pprint
import ray

ray.init()

2022-08-25 10:09:51,282	INFO worker.py:1223 -- Using address localhost:9031 set in the environment variable RAY_ADDRESS
2022-08-25 10:09:51,697	INFO worker.py:1333 -- Connecting to existing Ray cluster at address: 172.31.80.117:9031...
2022-08-25 10:09:51,706	INFO worker.py:1509 -- Connected to Ray cluster. View the dashboard at https://session-i8ddtfaxhwypbvnyb9uzg7xs.i.anyscaleuserdata-staging.com/auth/?token=agh0_CkcwRQIhAJXwvxwq31GryaWthvXGCXZebsijbuqi7qL2pCa5uROOAiBGjzsyXAJFHLlaEI9zSlNI8ewtghKg5UV3t8NmlxuMcRJmEiCtvjcKE0VPiU7iQx51P9oPQjfpo5g1RJXccVSS5005cBgCIgNuL2E6DAj9xazjBhDwj4veAUIMCP3ClJgGEPCPi94B-gEeChxzZXNfaThERFRmQVhId1lwYlZueWI5dVpnN3hT&redirect_to=dashboard 
2022-08-25 10:09:51,709	INFO packaging.py:342 -- Pushing file package 'gcs://_ray_pkg_3332f64b0a461fddc20be71129115d0a.zip' (0.34MiB) to Ray cluster...
2022-08-25 10:09:51,714	INFO packaging.py:351 -- Successfully pushed file package 'gcs://_ray_pkg_3332f64b0a461fddc20be71129115d0a.zip'.


Python version:,3.8.5
Ray version:,2.0.0
Dashboard:,http://session-i8ddtfaxhwypbvnyb9uzg7xs.i.anyscaleuserdata-staging.com/auth/?token=agh0_CkcwRQIhAJXwvxwq31GryaWthvXGCXZebsijbuqi7qL2pCa5uROOAiBGjzsyXAJFHLlaEI9zSlNI8ewtghKg5UV3t8NmlxuMcRJmEiCtvjcKE0VPiU7iQx51P9oPQjfpo5g1RJXccVSS5005cBgCIgNuL2E6DAj9xazjBhDwj4veAUIMCP3ClJgGEPCPi94B-gEeChxzZXNfaThERFRmQVhId1lwYlZueWI5dVpnN3hT&redirect_to=dashboard


We can check the resources our cluster is composed of. If you are running this notebook on your local machine or Google Colab, you should see the number of CPU cores and GPUs available on the said machine.

In [3]:
pprint(ray.cluster_resources())

{'CPU': 208.0,
 'GPU': 16.0,
 'accelerator_type:T4': 4.0,
 'memory': 616693614180.0,
 'node:172.31.76.237': 1.0,
 'node:172.31.80.117': 1.0,
 'node:172.31.85.193': 1.0,
 'node:172.31.85.32': 1.0,
 'node:172.31.90.137': 1.0,
 'object_store_memory': 259318055729.0}


In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/). We will be running the training using [Ray AIR](https://docs.ray.io/en/latest/ray-air/getting-started.html).

You can change those two variables to control whether the training (which we will get to later) uses CPUs or GPUs, and how many workers should be spawned. Each worker will claim one CPU or GPU. Make sure not to request more resources than the resources present!

By default, we will run the training with one GPU worker.

In [4]:
use_gpu = True  # set this to False to run on CPUs
num_workers = 1  # set this to number of GPUs/CPUs you want to use

## Fine-tuning a model on a text classification task

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences. If you would like to learn more, refer to the [original notebook](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb).

Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [5]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on your model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [6]:
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

### Loading the dataset <a name="load"></a>

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions.

As Ray AIR doesn't provide integrations for 🤗 Datasets yet, we will simply run the normal 🤗 Datasets code to load the dataset from the Hub.

In [ ]:
from datasets import load_dataset

actual_task = "mnli" if task == "mnli-mm" else task
datasets = load_dataset("glue", actual_task)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation, and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

We will also need the metric. In order to avoid serialization errors, we will load the metric inside the training workers later. Therefore, now we will just define the function we will use.

In [8]:
from datasets import load_metric

def load_metric_fn():
    return load_metric('glue', actual_task)

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric).

### Preprocessing the data with Ray AIR <a name="preprocess"></a>

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers' `Tokenizer`, which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure that:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [10]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

For Ray AIR, instead of using 🤗 Dataset objects directly, we will convert them to [Ray Datasets](https://docs.ray.io/en/latest/data/dataset.html). Both are backed by Arrow tables, so the conversion is straightforward. We will use the built-in `ray.data.from_huggingface` function.

In [11]:
import ray.data

ray_datasets = ray.data.from_huggingface(datasets)
ray_datasets

{'train': Dataset(num_blocks=1, num_rows=8551, schema={sentence: string, label: int64, idx: int32}),
 'validation': Dataset(num_blocks=1, num_rows=1043, schema={sentence: string, label: int64, idx: int32}),
 'test': Dataset(num_blocks=1, num_rows=1063, schema={sentence: string, label: int64, idx: int32})}

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer than what the model selected can handle will be truncated to the maximum length accepted by the model.

We use a `BatchMapper` to create a Ray AIR preprocessor that will map the function to the dataset in a distributed fashion. It will run during training and prediction.

In [12]:
import pandas as pd
from ray.data.preprocessors import BatchMapper

def preprocess_function(examples: pd.DataFrame):
    # if we only have one column, we are inferring.
    # no need to tokenize in that case. 
    if len(examples.columns) == 1:
        return examples
    examples = examples.to_dict("list")
    sentence1_key, sentence2_key = task_to_keys[task]
    if sentence2_key is None:
        ret = tokenizer(examples[sentence1_key], truncation=True)
    else:
        ret = tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
    # Add back the original columns
    ret = {**examples, **ret}
    return pd.DataFrame.from_dict(ret)

batch_encoder = BatchMapper(preprocess_function, batch_format="pandas")

### Fine-tuning the model with Ray AIR <a name="train"></a>

Now that our data is ready, we can download the pretrained model and fine-tune it.

Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class.

We will not go into details about each specific component of the training (see the [original notebook](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb) for that). The tokenizer is the same as we have used to encoded the dataset before.

The main difference when using the Ray AIR is that we need to create our 🤗 Transformers `Trainer` inside a function (`trainer_init_per_worker`) and return it. That function will be passed to the `HuggingFaceTrainer` and will run on every Ray worker. The training will then proceed by the means of PyTorch DDP.

Make sure that you initialize the model, metric, and tokenizer inside that function. Otherwise, you may run into serialization errors.

Furthermore, `push_to_hub=True` is not yet supported. Ray will, however, checkpoint the model at every epoch, allowing you to push it to hub manually. We will do that after the training.

If you wish to use thrid party logging libraries, such as MLflow or Weights&Biases, do not set them in `TrainingArguments` (they will be automatically disabled) - instead, you should pass Ray AIR callbacks to `HuggingFaceTrainer`'s `run_config`. In this example, we will use MLflow.

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
name = f"{model_name}-finetuned-{task}"

def trainer_init_per_worker(train_dataset, eval_dataset = None, **config):
    print(f"Is CUDA available: {torch.cuda.is_available()}")
    metric = load_metric_fn()
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    args = TrainingArguments(
        name,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=config.get("learning_rate", 2e-5),
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=config.get("epochs", 2),
        weight_decay=config.get("weight_decay", 0.01),
        push_to_hub=False,
        disable_tqdm=True,  # declutter the output a little
        no_cuda=not use_gpu,  # you need to explicitly set no_cuda if you want CPUs
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    print("Starting training")
    return trainer

With our `trainer_init_per_worker` complete, we can now instantiate the `HuggingFaceTrainer`. Aside from the function, we set the `scaling_config`, controlling the amount of workers and resources used, and the `datasets` we will use for training and evaluation.

We specify the `MlflowLoggerCallback` inside the `run_config`, and pass the preprocessor we have defined earlier as an argument. The preprocessor will be included with the returned `Checkpoint`, meaning it will also be applied during inference.

In [14]:
from ray.train.huggingface import HuggingFaceTrainer
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.air.integrations.mlflow import MLflowLoggerCallback

trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
    datasets={"train": ray_datasets["train"], "evaluation": ray_datasets[validation_key]},
    run_config=RunConfig(
        callbacks=[MLflowLoggerCallback(experiment_name=name)],
        checkpoint_config=CheckpointConfig(num_to_keep=1, checkpoint_score_attribute="eval_loss", checkpoint_score_order="min"),
    ),
    preprocessor=batch_encoder,
)

Finally, we call the `fit` method to start training with Ray AIR. We will save the `Result` object to a variable so we can access metrics and checkpoints.

In [15]:
result = trainer.fit()

Trial name,status,loc,iter,total time (s),loss,learning_rate,epoch
HuggingFaceTrainer_c1ff5_00000,TERMINATED,172.31.90.137:947,2,200.217,0.3886,0,2


(RayTrainWorker pid=1114, ip=172.31.90.137) 2022-08-25 10:10:44,617	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=4]


(RayTrainWorker pid=1114, ip=172.31.90.137) Is CUDA available: True
(RayTrainWorker pid=1116, ip=172.31.90.137) Is CUDA available: True
(RayTrainWorker pid=1117, ip=172.31.90.137) Is CUDA available: True
(RayTrainWorker pid=1115, ip=172.31.90.137) Is CUDA available: True


(RayTrainWorker pid=1117, ip=172.31.90.137) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
(RayTrainWorker pid=1117, ip=172.31.90.137) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(RayTrainWorker pid=1117, ip=172.31.90.137) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(RayTrainWorker pid=1117, ip=172.31.90.137) Some weights of DistilBertForSequ

(RayTrainWorker pid=1114, ip=172.31.90.137) Starting training
(RayTrainWorker pid=1116, ip=172.31.90.137) Starting training
(RayTrainWorker pid=1117, ip=172.31.90.137) Starting training
(RayTrainWorker pid=1115, ip=172.31.90.137) Starting training


...

(RayTrainWorker pid=1114, ip=172.31.90.137) {'train_runtime': 174.4696, 'train_samples_per_second': 98.023, 'train_steps_per_second': 6.133, 'train_loss': 0.4661755713346963, 'epoch': 2.0}


(RayTrainWorker pid=1114, ip=172.31.90.137) 
(RayTrainWorker pid=1114, ip=172.31.90.137) 
(RayTrainWorker pid=1114, ip=172.31.90.137) Training completed. Do not forget to share your model on huggingface.co/models =)
(RayTrainWorker pid=1114, ip=172.31.90.137) 
(RayTrainWorker pid=1114, ip=172.31.90.137) 


Result for HuggingFaceTrainer_c1ff5_00000:
  _time_this_iter_s: 96.96447467803955
  _timestamp: 1661447637
  _training_iteration: 2
  date: 2022-08-25_10-13-57
  done: false
  epoch: 2.0
  eval_loss: 0.6215357184410095
  eval_matthews_correlation: 0.42957017514952434
  eval_runtime: 0.9956
  eval_samples_per_second: 273.204
  eval_steps_per_second: 5.022
  experiment_id: 592e02b25b254bd1a3743904313dc85b
  hostname: ip-172-31-90-137
  iterations_since_restore: 2
  learning_rate: 0.0
  loss: 0.3886
  node_ip: 172.31.90.137
  pid: 947
  should_checkpoint: true
  step: 1070
  time_since_restore: 200.21722102165222
  time_this_iter_s: 96.97664165496826
  time_total_s: 200.21722102165222
  timestamp: 1661447637
  timesteps_since_restore: 0
  train_loss: 0.4661755713346963
  train_runtime: 174.4696
  train_samples_per_second: 98.023
  train_steps_per_second: 6.133
  training_iteration: 2
  trial_id: c1ff5_00000
  warmup_time: 0.003858327865600586
  


Result for HuggingFaceTrainer_c1ff5_00000:
  _time_this_iter_s: 96.96447467803955
  _timestamp: 1661447637
  _training_iteration: 2
  date: 2022-08-25_10-13-57
  done: true
  epoch: 2.0
  eval_loss: 0.6215357184410095
  eval_matthews_correlation: 0.42957017514952434
  eval_runtime: 0.9956
  eval_samples_per_second: 273.204
  eval_steps_per_second: 5.022
  experiment_id: 592e02b25b254bd1a3743904313dc85b
  experiment_tag: '0'
  hostname: ip-172-31-90-137
  iterations_since_restore: 2
  learning_rate: 0.0
  loss: 0.3886
  node_ip: 172.31.90.137
  pid: 947
  should_checkpoint: true
  step: 1070
  time_since_restore: 200.21722102165222
  time_this_iter_s: 96.97664165496826
  time_total_s: 200.21722102165222
  timestamp: 1661447637
  timesteps_since_restore: 0
  train_loss: 0.4661755713346963
  train_runtime: 174.4696
  train_samples_per_second: 98.023
  train_steps_per_second: 6.133
  training_iteration: 2
  trial_id: c1ff5_00000
  warmup_time: 0.003858327865600586
  


2022-08-25 10:14:09,300	INFO tune.py:758 -- Total run time: 246.67 seconds (246.44 seconds for the tuning loop).


You can use the returned `Result` object to access metrics and the Ray AIR `Checkpoint` associated with the last iteration.

In [16]:
result

Result(metrics={'loss': 0.3886, 'learning_rate': 0.0, 'epoch': 2.0, 'step': 1070, 'eval_loss': 0.6215357184410095, 'eval_matthews_correlation': 0.42957017514952434, 'eval_runtime': 0.9956, 'eval_samples_per_second': 273.204, 'eval_steps_per_second': 5.022, 'train_runtime': 174.4696, 'train_samples_per_second': 98.023, 'train_steps_per_second': 6.133, 'train_loss': 0.4661755713346963, '_timestamp': 1661447637, '_time_this_iter_s': 96.96447467803955, '_training_iteration': 2, 'should_checkpoint': True, 'done': True, 'trial_id': 'c1ff5_00000', 'experiment_tag': '0'}, error=None, log_dir=PosixPath('/home/ray/ray_results/HuggingFaceTrainer_2022-08-25_10-10-02/HuggingFaceTrainer_c1ff5_00000_0_2022-08-25_10-10-04'))

### Tune hyperparameters with Ray AIR <a name="predict"></a>

If we would like to tune any hyperparameters of the model, we can do so by simply passing our `HuggingFaceTrainer` into a `Tuner` and defining the search space.

We can also take advantage of the advanced search algorithms and schedulers provided by Ray Tune. In this example, we will use an `ASHAScheduler` to aggresively terminate underperforming trials.

In [17]:
from ray import tune
from ray.tune import Tuner
from ray.tune.schedulers.async_hyperband import ASHAScheduler

tune_epochs = 4
tuner = Tuner(
    trainer,
    param_space={
        "trainer_init_config": {
            "learning_rate": tune.grid_search([2e-5, 2e-4, 2e-3, 2e-2]),
            "epochs": tune_epochs,
        }
    },
    tune_config=tune.TuneConfig(
        metric="eval_loss",
        mode="min",
        num_samples=1,
        scheduler=ASHAScheduler(
            max_t=tune_epochs,
        )
    ),
    run_config=RunConfig(
        checkpoint_config=CheckpointConfig(num_to_keep=1, checkpoint_score_attribute="eval_loss", checkpoint_score_order="min")
    ),
)

In [18]:
tune_results = tuner.fit()

Trial name,status,loc,trainer_init_conf...,iter,total time (s),loss,learning_rate,epoch
HuggingFaceTrainer_5654d_00000,TERMINATED,172.31.90.137:1729,2e-05,4,347.171,0.1958,0,4
HuggingFaceTrainer_5654d_00001,TERMINATED,172.31.76.237:1805,0.0002,1,95.2492,0.6225,0.00015,1
HuggingFaceTrainer_5654d_00002,TERMINATED,172.31.85.32:1322,0.002,1,93.7613,0.6463,0.0015,1
HuggingFaceTrainer_5654d_00003,TERMINATED,172.31.85.193:1060,0.02,1,99.3677,0.926,0.015,1


(RayTrainWorker pid=1789, ip=172.31.90.137) 2022-08-25 10:14:23,379	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=4]


(RayTrainWorker pid=1792, ip=172.31.90.137) Is CUDA available: True
(RayTrainWorker pid=1790, ip=172.31.90.137) Is CUDA available: True
(RayTrainWorker pid=1791, ip=172.31.90.137) Is CUDA available: True
(RayTrainWorker pid=1789, ip=172.31.90.137) Is CUDA available: True


(RayTrainWorker pid=1974, ip=172.31.76.237) 2022-08-25 10:14:29,354	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=4]


(RayTrainWorker pid=1977, ip=172.31.76.237) Is CUDA available: True
(RayTrainWorker pid=1976, ip=172.31.76.237) Is CUDA available: True
(RayTrainWorker pid=1975, ip=172.31.76.237) Is CUDA available: True
(RayTrainWorker pid=1974, ip=172.31.76.237) Is CUDA available: True


...

(RayTrainWorker pid=1789, ip=172.31.90.137) Saving model checkpoint to distilbert-base-uncased-finetuned-cola/checkpoint-2140
(RayTrainWorker pid=1789, ip=172.31.90.137) Configuration saved in distilbert-base-uncased-finetuned-cola/checkpoint-2140/config.json


(RayTrainWorker pid=1789, ip=172.31.90.137) {'eval_loss': 0.8064090609550476, 'eval_matthews_correlation': 0.5322860764824153, 'eval_runtime': 1.0006, 'eval_samples_per_second': 271.827, 'eval_steps_per_second': 4.997, 'epoch': 4.0}


(RayTrainWorker pid=1789, ip=172.31.90.137) Model weights saved in distilbert-base-uncased-finetuned-cola/checkpoint-2140/pytorch_model.bin
(RayTrainWorker pid=1789, ip=172.31.90.137) tokenizer config file saved in distilbert-base-uncased-finetuned-cola/checkpoint-2140/tokenizer_config.json
(RayTrainWorker pid=1789, ip=172.31.90.137) Special tokens file saved in distilbert-base-uncased-finetuned-cola/checkpoint-2140/special_tokens_map.json
(RayTrainWorker pid=1789, ip=172.31.90.137) 
(RayTrainWorker pid=1789, ip=172.31.90.137) 
(RayTrainWorker pid=1789, ip=172.31.90.137) Training completed. Do not forget to share your model on huggingface.co/models =)
(RayTrainWorker pid=1789, ip=172.31.90.137) 
(RayTrainWorker pid=1789, ip=172.31.90.137) 


(RayTrainWorker pid=1789, ip=172.31.90.137) {'train_runtime': 329.1948, 'train_samples_per_second': 103.902, 'train_steps_per_second': 6.501, 'train_loss': 0.34860724689804506, 'epoch': 4.0}
Result for HuggingFaceTrainer_5654d_00000:
  _time_this_iter_s: 98.92064905166626
  _timestamp: 1661448005
  _training_iteration: 4
  date: 2022-08-25_10-20-05
  done: true
  epoch: 4.0
  eval_loss: 0.8064090609550476
  eval_matthews_correlation: 0.5322860764824153
  eval_runtime: 1.0006
  eval_samples_per_second: 271.827
  eval_steps_per_second: 4.997
  experiment_id: cee1b96afcf344e89482e3c5e298a412
  hostname: ip-172-31-90-137
  iterations_since_restore: 4
  learning_rate: 0.0
  loss: 0.1958
  node_ip: 172.31.90.137
  pid: 1729
  should_checkpoint: true
  step: 2140
  time_since_restore: 347.1705844402313
  time_this_iter_s: 98.93785095214844
  time_total_s: 347.1705844402313
  timestamp: 1661448005
  timesteps_since_restore: 0
  train_loss: 0.34860724689804506
  train_runtime: 329.1948
  train_

2022-08-25 10:20:13,409	INFO tune.py:758 -- Total run time: 361.90 seconds (361.74 seconds for the tuning loop).


We can view the results of the tuning run as a dataframe, and obtain the best result.

In [19]:
tune_results.get_dataframe().sort_values("eval_loss")

,loss,learning_rate,epoch,step,eval_loss,eval_matthews_correlation,eval_runtime,eval_samples_per_second,eval_steps_per_second,_timestamp,...,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,config/trainer_init_config/epochs,config/trainer_init_config/learning_rate,logdir
1,0.6225,0.00015,1.0,535,0.649242,0.000000,1.0157,267.792,4.923,1661447759,...,1805,ip-172-31-76-237,172.31.76.237,95.249164,0,1,0.003661,4,0.00020,/home/ray/ray_results/HuggingFaceTrainer_2022-...
3,0.9260,0.01500,1.0,535,0.652943,0.000000,0.9428,288.510,5.303,1661447782,...,1060,ip-172-31-85-193,172.31.85.193,99.367746,0,1,0.004133,4,0.02000,/home/ray/ray_results/HuggingFaceTrainer_2022-...
2,0.6463,0.00150,1.0,535,0.658653,0.000000,0.9576,284.050,5.222,1661447764,...,1322,ip-172-31-85-32,172.31.85.32,93.761317,0,1,0.004533,4,0.00200,/home/ray/ray_results/HuggingFaceTrainer_2022-...
0,0.1958,0.00000,4.0,2140,0.806409,0.532286,1.0006,271.827,4.997,1661448005,...,1729,ip-172-31-90-137,172.31.90.137,347.170584,0,4,0.003702,4,0.00002,/home/ray/ray_results/HuggingFaceTrainer_2022-...


In [20]:
best_result = tune_results.get_best_result()

### Predict on test data with Ray AIR <a name="predict"></a>

You can now use the checkpoint to run prediction with `HuggingFacePredictor`, which wraps around [🤗 Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines). In order to distribute prediction, we use `BatchPredictor`. While this is not necessary for the very small example we are using (you could use `HuggingFacePredictor` directly), it will scale well to a large dataset.

In [21]:
from ray.train.huggingface import HuggingFacePredictor
from ray.train.batch_predictor import BatchPredictor
import pandas as pd

predictor = BatchPredictor.from_checkpoint(
    checkpoint=best_result.checkpoint,
    predictor_cls=HuggingFacePredictor,
    task="text-classification",
    device=0 if use_gpu else -1,  # -1 is CPU, otherwise device index
)
prediction = predictor.predict(ray_datasets["test"].map_batches(lambda x: x[["sentence"]]), num_gpus_per_worker=int(use_gpu))
prediction.show()

Map Progress (1 actors 1 pending): 100%|██████████| 1/1 [00:18<00:00, 18.46s/it]


{'label': 'LABEL_1', 'score': 0.6822417974472046}
{'label': 'LABEL_1', 'score': 0.6822402477264404}
{'label': 'LABEL_1', 'score': 0.6822407841682434}
{'label': 'LABEL_1', 'score': 0.6822386980056763}
{'label': 'LABEL_1', 'score': 0.6822428107261658}
{'label': 'LABEL_1', 'score': 0.6822453737258911}
{'label': 'LABEL_1', 'score': 0.6822437047958374}
{'label': 'LABEL_1', 'score': 0.6822428703308105}
{'label': 'LABEL_1', 'score': 0.6822431683540344}
{'label': 'LABEL_1', 'score': 0.6822426915168762}
{'label': 'LABEL_1', 'score': 0.6822447776794434}
{'label': 'LABEL_1', 'score': 0.6822456121444702}
{'label': 'LABEL_1', 'score': 0.6822471022605896}
{'label': 'LABEL_1', 'score': 0.6822477579116821}
{'label': 'LABEL_1', 'score': 0.682244598865509}
{'label': 'LABEL_1', 'score': 0.6822422742843628}
{'label': 'LABEL_1', 'score': 0.6822470426559448}
{'label': 'LABEL_1', 'score': 0.6822417378425598}
{'label': 'LABEL_1', 'score': 0.6822449564933777}
{'label': 'LABEL_1', 'score': 0.682239294052124}


### Share the model <a name="share"></a>

To be able to share your model with the community, there are a few more steps to follow.

We have conducted the training on the Ray cluster, but share the model from the local enviroment - this will allow us to easily authenticate.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Now, load the model and tokenizer locally, and recreate the 🤗 Transformers `Trainer`:

In [ ]:
from ray.train.huggingface import HuggingFaceCheckpoint

checkpoint = HuggingFaceCheckpoint.from_checkpoint(result.checkpoint)
hf_trainer = checkpoint.get_model(model=AutoModelForSequenceClassification)

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
hf_trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("sgugger/my-awesome-model")
```